In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sqlalchemy import create_engine
import streamlit as st
from dotenv import load_dotenv
import os

ModuleNotFoundError: No module named 'dotenv'

In [ ]:
load_dotenv()

In [ ]:
import pandas as pd
import psycopg2

# Database connection
conn = psycopg2.connect(
    dbname=os.getenv("DB_NAME"), user=os.getenv("USERNAME"), password= os.getenv("PASSWORD"), host= os.getenv("HOST_NAME"), port= os.getenv("PORT")
)
query = "SELECT * FROM table"
df = pd.read_sql_query(query, conn)
conn.close()

# Quick look at the dataset
print(df.info())
print(df.describe())


In [ ]:
df.fillna(df.mean(), inplace=True)


In [ ]:
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1

# Filter outliers
df = df[~((df < (Q1 - 1.5 * IQR)) | (df > (Q3 + 1.5 * IQR))).any(axis=1)]


In [ ]:
# Descriptive statistics
print(df.describe())

# Variable summary
print(df.info())


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Histogram for session duration
plt.hist(df['session_duration'], bins=30, alpha=0.7)
plt.title("Distribution of Session Duration")
plt.xlabel("Duration (seconds)")
plt.ylabel("Frequency")
plt.show()


In [ ]:
corr = df.corr()
sns.heatmap(corr, annot=True, cmap='coolwarm')
plt.title("Correlation Matrix")
plt.show()


In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaled_metrics = scaler.fit_transform(df[['sessions_frequency', 'session_duration', 'total_traffic']])
df_scaled = pd.DataFrame(scaled_metrics, columns=['sessions_frequency', 'session_duration', 'total_traffic'])


In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# Elbow method
inertia = []
k_range = range(1, 11)

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(df_scaled)
    inertia.append(kmeans.inertia_)

# Plot the elbow curve
plt.plot(k_range, inertia, marker='o')
plt.title("Elbow Method for Optimal k")
plt.xlabel("Number of Clusters")
plt.ylabel("Inertia")
plt.show()



In [ ]:
optimal_k = 3  # Replace with the chosen k from the elbow method
kmeans = KMeans(n_clusters=optimal_k, random_state=42)
df['cluster'] = kmeans.fit_predict(df_scaled)


In [ ]:
cluster_summary = df.groupby('cluster').agg({
    'sessions_frequency': ['min', 'max', 'mean', 'sum'],
    'session_duration': ['min', 'max', 'mean', 'sum'],
    'total_traffic': ['min', 'max', 'mean', 'sum']
})
print(cluster_summary)


#### Top 10 Handsets

In [ ]:
top_10_handsets = df['handset'].value_counts().head(10)
print(top_10_handsets)


#### Top 3 Manufacturers

In [ ]:
top_manufacturers = df['manufacturer'].value_counts().head(3)
print(top_manufacturers)

#### Top 5 Handsets per Manufacturer:

In [ ]:
for manufacturer in top_manufacturers.index:
    print(f"Top 5 handsets for {manufacturer}:")
    print(df[df['manufacturer'] == manufacturer]['handset'].value_counts().head(5))


##### Aggregate metrics per user

In [ ]:
app_columns = ['social_media_sessions', 'google_sessions', 'email_sessions', 
               'youtube_sessions', 'netflix_sessions', 'gaming_sessions', 'other_sessions']

df_user_agg = df.groupby('user_id').agg({
    'xDR_sessions': 'count',
    'session_duration': 'sum',
    'DL_data': 'sum',
    'UL_data': 'sum'
})

df_user_agg['total_data'] = df_user_agg['DL_data'] + df_user_agg['UL_data']
print(df_user_agg.head())


##### Missing values and Outliers

In [ ]:
df.fillna(df.mean(), inplace=True)


#Handled via IQR
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1
df = df[~((df < (Q1 - 1.5 * IQR)) | (df > (Q3 + 1.5 * IQR))).any(axis=1)]


#### Reporting Metrics

##### Variables and Types

###### Variable Transformations

In [ ]:
df['duration_decile'] = pd.qcut(df['session_duration'], 10, labels=False)
decile_agg = df.groupby('duration_decile').agg({'total_data': 'sum'})
print(decile_agg)


###### Non-Graphical Univariate Analysis

In [ ]:
df[['xDR_sessions', 'session_duration', 'DL_data', 'UL_data']].describe()


###### Graphical Univariate Analysis

###### Bivariate Analysis

In [ ]:
sns.scatterplot(x='google_data', y='total_data', data=df)

#### Correlation Analysis

In [ ]:
app_columns = ['social_media_data', 'google_data', 'email_data', 'youtube_data', 'netflix_data', 'gaming_data', 'other_data']
corr_matrix = df[app_columns].corr()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')


#### Dimensionality Reduction

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
pca_result = pca.fit_transform(df[app_columns])
print(f"Explained variance ratio: {pca.explained_variance_ratio_}")
